In [1]:
import pandas as pd
import numpy as np
from collections import deque

# Goal API of Tree

In [2]:
df = pd.read_csv("file.csv")
train, test = train_test_split(df, test_size=0.2) # find actual one to use


my_tree = DecisionTree()
my_tree.fit(X, Y) # X being all data, Y being just labels for each sample
my_tree.predict(X) #---> return an array of target labels for each i in X (find shape of X at first)
my_tree.score(X, y[, sample_weight]) # Return the mean accuracy on the given test data and labels.

my_tree.plot() # returns graph visual of nodes
my_tree.depth() # return max depth


SyntaxError: invalid syntax (<ipython-input-2-ea7206607fc0>, line 8)

### General Notes:

- How to handle a column that is not categorical? That is numerical
- Create Method to determine type of column.
- Change fit to take in X vectors and Y target vector
- Convert to some other python tree for easy printing or something. Grahpical
- Incorportate the Deku Tree for site

### Site Posting Notes:
- Use stanford kids site as rough guide but make cooler with Guess who. Start with that.
- Note I chose more verbose programming statements in key parts of the algorithm as 1. More clear what part of the algo I'm doing. This is for me to learn, explain, and for others to easily see. If this were production I would be more likely to use more efficient python one liners. Have an example of a crazy one liner of like gini impurity.

- For each method, have latex like 436 of math algorithm.

In [30]:
class DecisionNode:
    #  Class attribute. Same value for every instance of this class.
    numMoney = 100000

    def __init__(self, num_samples):
        self.num_samples = num_samples
        
        self.split_feature = None
        self.split_value = None
        self.gini = None
        self.left_child = None 
        self.right_child = None
        self.class_label = None 
        self.is_leaf = False

    def __str__(self):
        return f"NumSamples: {self.num_samples}" \
        f"\nLabel: {self.class_label}" \
        f"\nGini: {self.gini}" \
        f"\nColumnToSplit: {self.split_feature}" \
        f"\nSplitValue: {self.split_value}" \
        f"\nLeaf: {self.is_leaf}\n"

In [102]:
class DecisionTree:

    def __init__(self, max_depth):
        self.root_node = None
        self.num_nodes = 0
        self.target_feature = None
        self.max_depth = max_depth
        self.node_count = 0

    def predict(self, sample):
        """Predict the class label of the given sample."""
        node = self.root_node
        
        while(node.is_leaf == False):
            
            if sample[node.split_feature] == node.split_value:
                node = node.left_child
            else:
                node = node.right_child
        return node.class_label

    def _gini(self, df):
        """Returns gini score of df
        uses target_feature variable of tree to find score
        takes in a DF right now.
        
        
        """
        t = len(df)
        p = df.groupby(self.target_feature).size() / t
        return 1 - sum(p**2)

    def _impurity(self, df, prop):
        s = 0
        for k, subf in df.groupby(prop):
            g = self._gini(subf)

            # Get weighted average of gini values for a whole columns different values.
            s += g * (len(subf) / len(df))
        return s

    def _split_feature(self, df):
        """Find the best feature to split on given a df. Return column to split on"""
        # make better with just two lists? or a list of lists?
        col_scores = {}
        data = df.drop(self.target_feature, axis=1)

        for column in data.columns:
            weighted_gini = self._impurity(df, column)
            col_scores[column] = weighted_gini

        # return col_scores
        # return (min(col_scores, key=col_scores.get))

        lowest_feature = min(col_scores, key=col_scores.get)
        lowest_score = col_scores[lowest_feature]

        return lowest_feature, lowest_score

    def _split_value(self, df, feature):
        """Find the best value of a given feature to split on"""

        values = {}
        for k, subf in df.groupby(feature):
            g = self._gini(subf)
            # Weighted average of gini score.
            values[k] = g * len(subf) / len(df)

        # return values
        # return value that has the lowest gini score
        return (min(values, key=values.get))

    def fit(self, df, target):
        """where target is string of target column to predict."""
        self.target_feature = target
        root_node = DecisionNode(len(df))

    def _build_tree(self, df):
        self.root_node = DecisionNode(len(df))

        # Recursively add nodes to tree
        self._splitNode(df, self.root_node, 0)

    def _splitNode(self, df, node, current_level):
        """Given a df, split that into two subframes based on the best gini-gain"""
        self.node_count += 1

        feature, feature_score = self._split_feature(df)
        node.gini = self._gini(df)
        node.class_label = df[self.target_feature].mode()[0]


      #  weighted_impurity_decrease = (
       #     node.num_samples / self.root_node.num_samples) * (node.gini - feature_score)

        # if ((node.gini - feature_score) > self.threshold ):

        # if(node.gini > feature_score):
        if(current_level < self.max_depth):
            current_level += 1

            node.is_leaf = False
            node.split_feature = feature
            node.split_value = self._split_value(df, feature)

            left_df = df[df[feature] == node.split_value]  # left == true
            right_df = df[df[feature] != node.split_value]  # right == false
            
            if(len(left_df) > 0):
                node.left_child = DecisionNode(len(left_df))
                self._splitNode(left_df, node.left_child, current_level)
            if(len(right_df) > 0):
                node.right_child = DecisionNode(len(right_df))
                self._splitNode(right_df, node.right_child, current_level)
            return
        else:
            node.is_leaf = True
            return

    def score(self, test):
        """Return accuracy on given samples and labels"""
        result = pd.Series(dtype=object)
        results = test.apply(lambda x: True if x[self.target_feature] == self.predict(x) else False, axis=1)
        score = ( results.value_counts()[True] / len(results) ) * 100
        return score
      

    def print_nodes(self):
        """Print tree in level order"""
        if self.root_node == None:
            return

        queue = deque()
        queue.append(self.root_node)

        while(len(queue) > 0):
            node = queue.popleft()
            print(node)

            if node.left_child != None:
                queue.append(node.left_child)
            if node.right_child is not None:
                queue.append(node.right_child)

    # if category is numerical, split into quartiles, get scores of each chunk?

## New Draft below:
# CURRENT OBJECTIVE:
* Gini method with numpy DONE
* Impurity method with numpy 
  
  
### New Gini Draft

In [1]:
import numpy as np
class DecisionNode:

    def __init__(self, num_samples):
        self.num_samples = num_samples
        
        self.split_feature = None
        self.split_value = None
        self.gini = None
        self.left_child = None 
        self.right_child = None
        self.class_label = None 
        self.is_leaf = False

    def __str__(self):
        return f"NumSamples: {self.num_samples}" \
        f"\nLabel: {self.class_label}" \
        f"\nGini: {self.gini}" \
        f"\nColumnToSplit: {self.split_feature}" \
        f"\nSplitValue: {self.split_value}" \
        f"\nLeaf: {self.is_leaf}\n"

In [24]:
class DecisionTree:

    def __init__(self, max_depth):
        self.root_node = None
        self.num_nodes = 0
        self.max_depth = max_depth
        self.node_count = 0

    def fit(self, X, y):
        """
        fit builds the decision tree from the given training samples and their 
        respective labels

        : array X: An array of training input samples of shape(n_samples, n_features)
        : array y: An array representing the labels for each sample of shape(n_samples)

        """

    def gini_impurity(self, y):
        """
        Returns the gini score of the array 
        TODO: THIS WORKS. WAS TESTED.  
        1 FEEDS TO IMPURITY NEXT ->

        : array y: Array containing labels
        """
        n = len(y)
        # labels: array containing all unique labels in y
        # freq: where freq[i] = frequency of lables[i]
        labels, freq = np.unique(y, return_counts=True)

        # prob: performs vector wise division. Is a vector
        prob = freq / n

        # Vector wise squaring of the probabilites
        prob = prob**2
        # Add them all up
        sigma_prob = prob.sum()

        gini = 1 - sigma_prob
        return gini


    def category_impurity(self, split_feature, y_labels):
        """
        Finds the total gini impurity for a possible split on the given vector.

        :np.array split_feature: Vector of categorical feature values
        :np.array y_labels: Vector of labels corresponding to split_feature
        """
        n = len(y_labels)
        # If I divide x_feature column up into groups for each of its different values, what is the gini score for each one
        total_impurity = 0

        # All the unique values of the proposed feature column
        feature_values = np.unique(split_feature)

        # For each  unique value in the feature column, find the weighted gini impurity of dividing the sample space
        # into groups of that value, return the total impurity for all groups.

        for value in feature_values:
            y_group = y_labels[np.where(split_feature == value)]

            gini_val = self.gini_impurity(y_group)

            weighted_impurity = gini_val * (len(y_group) / n)

            total_impurity += weighted_impurity

        return total_impurity

## CURRENT METHOD WORKING ON
- Will i need 2 versions of this for categorical and numerical? or have boolean option for that
- How to find the total gini impurity for a feature column if the values are NUMERICAL!!!

In [ ]:
def numerical_impurity(split_feature, y_labels):
    

In [25]:
x = np.array(['cat', 'cat', 'dog'])
y = np.array([20, 30, 0])

z = y[np.where(x == 'cat')]
z

array([20, 30])

In [47]:
initial = np.array(['red', 'red', 'red', 'blue', 'blue', 'blue', 'green', 'green', 'green'])
data = np.array([1, 2, 2,2,2,2,2,2,2])
total = gini_impurity(initial)

possible = category_impurity(data, initial)

print(f"Gini Gain: {total - possible}")

Gini Gain: 0.08333333333333348


## OLD VERSION OF METHOD:

In [48]:
# Do this but with numerical data.
def category_impurity(split_feature, y_labels):
    """
    Finds the total gini impurity for a possible split on the given vector.
    
    :np.array split_feature: Vector of categorical feature values
    :np.array y_labels: Vector of labels corresponding to split_feature
    """
    n = len(y_labels)
    # If I divide x_feature column up into groups for each of its different values, what is the gini score for each one
    total_impurity = 0
    
    # All the unique values of the proposed feature column
    feature_values = np.unique(split_feature)
    
    # For each  unique value in the feature column, find the weighted gini impurity of dividing the sample space 
    # into groups of that value, return the total impurity for all groups.
    
    for value in feature_values:
        y_group = y_labels[np.where(split_feature == value)]
        
        gini_val = gini_impurity(y_group)
        
        weighted_impurity = gini_val * (len(y_group) / n )
        
        total_impurity += weighted_impurity
        
    return total_impurity

# Tips

In [45]:
## documentation
def test_function(p1, p2, p3):
    """
    test_function does blah blah blah.

    :param p1: describe about parameter p1
    :param p2: describe about parameter p2
    :param p3: describe about parameter p3
    :return: describe what it returns
    """ 

In [ ]:
# very good query, like sql
df_low = df.loc[df["salary"]<6000,"salary"]

# Lambda usuage
df['age']=df.apply(lambda x: x['age']+3,axis=1)

#Get a single value. Goes [row][column]
sub_df.iloc[0]['A']

# lambda
efficiency_df["Speed Multiplier"] = efficiency_df.apply(lambda x: x["Time(s)"] / efficiency_df.iloc[0]["Time(s)"], axis=1)



In [ ]:
def entropy(self, df):
    t = len(df)
    p = df.groupby('class').size() / t
    return sum(p * np.log2(p.values))


def gini(self, df):  # performs gini on just one set, the 'class' column of mushroom df. scikit does not like strings
    # scikit does not like categorical, must turn to enumeration if column is a string.
    # this is the gini score if were to split this df on just this feature. again, lower is better.
    t = len(df)
    # should return series of length for each category. Thats why
    # here class is a column of X, so groupby column index?

    # group all columns of X by y-target value, then count them, then divide by the length of total X columns.
    # this returns an ARRAY of values.

    # count all y-column by how many x-rows have that certain value, count that.
    p = df.groupby('class').size() / t
    # sum is used below.
    return 1 - sum(p**2)


# returns the unique values for this categorical column.
df.groupby('bruises').groups.keys()

# how many samples of each category for feature 'odor'
df.groupby('odor').size()

for k, subf in df.groupby('bruises'):
    # returns a subframe for each category of bruises values.
    print(k, subf)

# Here, we see if splitting the df by a certain feature helps get a better gini score
# in relation to the 'class' variable, which is what we are trying to predict.
for k, subf in df.groupby('cap-shape'):
    # outputs 6 gini scores, how do we know 'cap-shape' is better than like another feature with 8 gini scores?
    # best way is take a 'weighted-average' of values.
    print(k, gini(subf))

# weighted average for a WHOLE column.
# returns the information gain to answering question of split in gini fxn. how can we best split to better gain info.
s = 0
for k, subf in df.groupby('cap-shape'):

    # score of this one feature split, trying to predicy 'y' class val.
    g = gini(subf)
    # ^ this is score for exactly one variable
    s += g * len(subf) / len(df)
    # this is the 'weighted average' of the gini score, takes into effect how large that split is. how many elements
    #
display(s)

# ^if this is smaller than the original parent nodes gini score,
# (the default score of just the gini function) then that means we are making progress
# no, just do pandas for rn. do numpy later once working.


def gini_numpy(self, X, y, y_labels):
    count = 0
    for i in range(len(y)):

In [ ]:
# refined code of above. maybe have a parameter for which scoring fxn to use?
# this is biased towards higher cardinality sets. just by definition of gini and dividing
# algorithm "c.45", sea.45?: ratio of information gain/ entropy of entire dataset. helps bias back towards lower cardinality sets.
# this is an example of a greedy algorithm. Finding best option at this current point.
# At each node, its a local optimization. might not be the best. Greedy.

#this only does the impurity for ONE FEATURE(column).
def impurity(df, prop):
    s = 0
    for k, subf in df.groupby(prop):
        # score of this one feature split, trying to predicy 'y' class val.
        g = gini(subf)
        # ^ this is score for exactly one variable
        s += g * len(subf) / len(df) # Get weighted average of gini values for a whole columns different values.
    return s

impurity(df, 'odor')

# skip first one, thats our target class.
# make this a recursive process( or stack somehow? maybe two stacks, one for node and one for list indices)
# if continuous data, split into buckets.

# Given a df/sub-df, this will find the impurities for EACH column, if we were to split on that column somehow.
# Then we can find the one with the lowest, and split on that. 
# ?? if that column has many categories though, we should go to the lowest gini score of that selected column, right?
# maybe if two classes are the same gini score, go with the one that is bigger? idk. or just choose first.
s = []
for f in fields[1:]: #MAKE sure to not grab label-column
    # sort these values to always choose LOWEST value. if it is zero that is perfect.
    s.append((f, impurity(df, f)))
    print(f, impurity(df, f))
    
# ORIGINAL WEIGHTED GINI IMPURITY - NEW SPLIT WEIGHTED AVERAGE GINI = GINI-GAIN( how much impurity we have removed)
# we want to maximize the gini-gain, how much that split better gives info. the best new split will always be the 
# lowest value possible( if zero, it means we have removed ALL the impurity by from dataset by splitting this way.)